# Исследование надежности заемщиков.

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.
Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.


Описание данных
* children — количество детей в семье
* days_employed — общий трудовой стаж в днях
* dob_years — возраст клиента в годах
* education — уровень образования клиента
* education_id — идентификатор уровня образования
* family_status — семейное положение
* family_status_id — идентификатор семейного положения
* gender — пол клиента
* income_type — тип занятости
* debt — имел ли задолженность по возврату кредитов
* total_income — ежемесячный доход
* purpose — цель получения кредита

## Шаг 1. Обзор данных

Выведем первые строки таблицы и общую информацию о ней

In [1]:
import pandas as pd

In [2]:
pth1 = 'data.csv'
pth2 = '/datasets/data.csv'

In [3]:
try:
    data = pd.read_csv(pth1)
except:
    data = pd.read_csv(pth2)

In [4]:
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [5]:
data.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Итак, в таблице 12 столбцов. В названиях колонок стилистических нарушений или ошибок нет. Количество значений в стобцах различается, значит, есть пропущенные значения. Также в столбце days_employed присутствуют как положительные, так и отрицательные значения, чего быть не должно. Также не похоже, чтобы эти значения были выражены в днях, скорее, в часах. Значения в столбце education написаны разным регистром.

 ## Шаг 2  Предобработка данных

###  Заполнение пропусков

Определим количество пропущенных значений и выведем строки с ними

In [6]:
data.isna().sum() 

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

In [7]:
data[data['days_employed'].isna()].head(10) 

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу
65,0,NaN,21,среднее,1,Не женат / не замужем,4,M,компаньон,0,NaN,операции с коммерческой недвижимостью
67,0,NaN,52,высшее,0,женат / замужем,0,F,пенсионер,0,NaN,покупка жилья для семьи
72,1,NaN,32,высшее,0,женат / замужем,0,M,госслужащий,0,NaN,операции с коммерческой недвижимостью
82,2,NaN,50,высшее,0,женат / замужем,0,F,сотрудник,0,NaN,жилье
83,0,NaN,52,среднее,1,женат / замужем,0,M,сотрудник,0,NaN,жилье


In [8]:
data.isna().mean()

children            0.000000
days_employed       0.100999
dob_years           0.000000
education           0.000000
education_id        0.000000
family_status       0.000000
family_status_id    0.000000
gender              0.000000
income_type         0.000000
debt                0.000000
total_income        0.100999
purpose             0.000000
dtype: float64

Пропуски обнаружены в столбцах "общий трудовой стаж" и "ежемесячный доход" и они соотвуетствуют друг другу. Доля пропущенных значений составляет около 10%. Вряд ли пропуски могут быть вызваны реальным отсутствием стажа, и соответственно дохода, у заемщиков, учитывая их тип занятости или тем, что заемщики намеренно не указывали эти данные, так как это все-таки необходимо для получения кредита. Так что, возможно, это техническая ошибка

Далее заполним выявленные пропуски медианным значением по столбцу.
Используем медиану вместо среднего, так как данные сильно отличаются между собой, и среднее значение не будет показательно.

In [9]:
data['days_employed'].median()

-1203.369528770489

In [10]:
data['total_income'].median()

145017.93753253992

In [11]:
median_income = data.groupby('income_type')['total_income'].median()
median_income

income_type
безработный        131339.751676
в декрете           53829.130729
госслужащий        150447.935283
компаньон          172357.950966
пенсионер          118514.486412
предприниматель    499163.144947
сотрудник          142594.396847
студент             98201.625314
Name: total_income, dtype: float64

In [12]:
data.loc[(data['total_income'].isna()) & (data['income_type'] == 'безработный'), 'total_income'] = median_income[0]
data.loc[(data['total_income'].isna()) & (data['income_type'] == 'в декрете'), 'total_income'] = median_income[1]
data.loc[(data['total_income'].isna()) & (data['income_type'] == 'госслужащий'), 'total_income'] = median_income[2]
data.loc[(data['total_income'].isna()) & (data['income_type'] == 'компаньон'), 'total_income'] = median_income[3]
data.loc[(data['total_income'].isna()) & (data['income_type'] == 'пенсионер'), 'total_income'] = median_income[4]
data.loc[(data['total_income'].isna()) & (data['income_type'] == 'предприниматель'), 'total_income'] = median_income[5]
data.loc[(data['total_income'].isna()) & (data['income_type'] == 'сотрудник'), 'total_income'] = median_income[6]
data.loc[(data['total_income'].isna()) & (data['income_type'] == 'студент'), 'total_income'] = median_income[7]

In [13]:
median_days = data.groupby('income_type')['days_employed'].median()
median_days

income_type
безработный        366413.652744
в декрете           -3296.759962
госслужащий         -2689.368353
компаньон           -1547.382223
пенсионер          365213.306266
предприниматель      -520.848083
сотрудник           -1574.202821
студент              -578.751554
Name: days_employed, dtype: float64

In [14]:
data.loc[(data['days_employed'].isna()) & (data['income_type'] == 'безработный'), 'days_employed'] = median_days[0]
data.loc[(data['days_employed'].isna()) & (data['income_type'] == 'в декрете'), 'days_employed'] = median_days[1]
data.loc[(data['days_employed'].isna()) & (data['income_type'] == 'госслужащий'), 'days_employed'] = median_days[2]
data.loc[(data['days_employed'].isna()) & (data['income_type'] == 'компаньон'), 'days_employed'] = median_days[3]
data.loc[(data['days_employed'].isna()) & (data['income_type'] == 'пенсионер'), 'days_employed'] = median_days[4]
data.loc[(data['days_employed'].isna()) & (data['income_type'] == 'предприниматель'), 'days_employed'] = median_days[5]
data.loc[(data['days_employed'].isna()) & (data['income_type'] == 'сотрудник'), 'days_employed'] = median_days[6]
data.loc[(data['days_employed'].isna()) & (data['income_type'] == 'студент'), 'days_employed'] = median_days[7]

In [15]:
data.isna().sum() #снова проверим количество пропусков

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### Проверка данных на аномалии и исправления.

In [17]:
data.groupby('income_type')['days_employed'].mean()

income_type
безработный        366413.652744
в декрете           -3296.759962
госслужащий         -3328.308350
компаньон           -2055.165652
пенсионер          365025.963652
предприниматель      -520.848083
сотрудник           -2251.736421
студент              -578.751554
Name: days_employed, dtype: float64

Вывели среднее значение трудовых дней стажа по типам занятости. Как уже было отмечено ранее, скорее всего значения представлены в часах. 

Избавимся от минусов и переведем в дни

In [20]:
data['days_employed'] = data['days_employed'].apply(lambda x: x if x<0 else x/24)
data['days_employed'] = data['days_employed'].abs()
data.groupby('income_type')['days_employed'].mean()

income_type
безработный        15267.235531
в декрете           3296.759962
госслужащий         3328.308350
компаньон           2055.165652
пенсионер          15209.415152
предприниматель      520.848083
сотрудник           2251.736421
студент              578.751554
Name: days_employed, dtype: float64

Теперь данные представлены в днях, но все равно выглядят нереалистично, так как странен такой разрыв стажа безработного и пенсионеров и остальных категорий. Вероятно, это ошибка при внесении данных

In [22]:
data['dob_years'].sort_values().unique()

array([ 0, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51,
       52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68,
       69, 70, 71, 72, 73, 74, 75], dtype=int64)

In [23]:
data['gender'].unique()

array(['F', 'M', 'XNA'], dtype=object)

В столбцах возраст и пол так же встречаются аномальные значения. Столбцы, в которых выявлены аномалии, не участвуют в исследовании, поэтому мы не исправляем их.

In [24]:
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5], dtype=int64)

In [25]:
data['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

В колонке "количество детей" есть значения -1 и 20. Отрицательного значения здесь не может быть. 20, скорее всего, тоже опечатка, так как такое количетсво детей все-таки редкость :), а в таблице людей с таким количеством детей 76, и, возможно, имелось в виду 2 ребенка.

In [26]:
data['children'] = data['children'].abs()
data['children'] = data['children'].replace(20, 2)

In [27]:
data['children'].value_counts()

0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64

### Изменение типов данных.

Заменим вещественный тип данных в столбце total_income на целочисленный

In [28]:
data['total_income'] = data['total_income'].astype('int')

In [29]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     21525 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  int32  
 11  purpose           21525 non-null  object 
dtypes: float64(1), int32(1), int64(5), object(5)
memory usage: 1.9+ MB


### Удаление дубликатов.

Посмотрим сколько всего дубликатов и какие они

In [30]:
data.duplicated().sum()

54

In [31]:
data[data.duplicated(keep=False) == True].sort_values(by='purpose').head(20)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
13639,0,15217.221094,64,среднее,1,женат / замужем,0,F,пенсионер,0,118514,автомобиль
8178,0,15217.221094,64,среднее,1,женат / замужем,0,F,пенсионер,0,118514,автомобиль
120,0,1574.202821,46,среднее,1,женат / замужем,0,F,сотрудник,0,142594,высшее образование
16378,0,1574.202821,46,среднее,1,женат / замужем,0,F,сотрудник,0,142594,высшее образование
20702,0,15217.221094,64,среднее,1,женат / замужем,0,F,пенсионер,0,118514,дополнительное образование
8583,0,15217.221094,58,высшее,0,Не женат / не замужем,4,F,пенсионер,0,118514,дополнительное образование
12389,0,15217.221094,64,среднее,1,женат / замужем,0,F,пенсионер,0,118514,дополнительное образование
1511,0,15217.221094,58,высшее,0,Не женат / не замужем,4,F,пенсионер,0,118514,дополнительное образование
3609,0,15217.221094,64,среднее,1,женат / замужем,0,F,пенсионер,0,118514,жилье
18428,0,15217.221094,64,среднее,1,женат / замужем,0,F,пенсионер,0,118514,жилье


In [32]:
#Выявленные дубликаты таковыми не являются, значит, удалять их не стоит

Найдем неявные дубликаты и избавимся от них

Используется метод unique для поиска уникальных значений в столбцах

In [33]:
data['education'].unique()

array(['высшее', 'среднее', 'Среднее', 'СРЕДНЕЕ', 'ВЫСШЕЕ',
       'неоконченное высшее', 'начальное', 'Высшее',
       'НЕОКОНЧЕННОЕ ВЫСШЕЕ', 'Неоконченное высшее', 'НАЧАЛЬНОЕ',
       'Начальное', 'Ученая степень', 'УЧЕНАЯ СТЕПЕНЬ', 'ученая степень'],
      dtype=object)

In [34]:
data['education'] = data['education'].str.lower() #приводим все значения к нижнему регистру

In [35]:
data['education'].unique() #проверяем

array(['высшее', 'среднее', 'неоконченное высшее', 'начальное',
       'ученая степень'], dtype=object)

In [36]:
data['family_status'].unique()

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'Не женат / не замужем'], dtype=object)

In [37]:
data['family_status'] = data['family_status'].str.lower()
data['family_status'].unique()

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'не женат / не замужем'], dtype=object)

In [38]:
data['purpose'].sort_values().unique()

array(['автомобили', 'автомобиль', 'высшее образование',
       'дополнительное образование', 'жилье',
       'заняться высшим образованием', 'заняться образованием',
       'на покупку автомобиля', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля', 'на проведение свадьбы',
       'недвижимость', 'образование', 'операции с жильем',
       'операции с коммерческой недвижимостью',
       'операции с недвижимостью', 'операции со своей недвижимостью',
       'покупка жилой недвижимости', 'покупка жилья',
       'покупка жилья для сдачи', 'покупка жилья для семьи',
       'покупка коммерческой недвижимости', 'покупка недвижимости',
       'покупка своего жилья', 'получение высшего образования',
       'получение дополнительного образования', 'получение образования',
       'приобретение автомобиля', 'профильное образование',
       'ремонт жилью', 'свадьба', 'свой автомобиль',
       'сделка с автомобилем', 'сделка с подержанным автомобилем',
       'строительство 

In [39]:
data['purpose'] = data['purpose'].replace('ремонт жилью', 'ремонт жилья') #исправили опечатку

После исправления неявных дубликатов, удалим выявленные дубликаты и проверим, что они удалены

In [40]:
data = data.drop_duplicates().reset_index()

In [41]:
data.duplicated().sum()

0

###  Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма.

In [42]:
education_dict = data[['education', 'education_id']]
education_dict.drop_duplicates()

,education,education_id
0,высшее,0
1,среднее,1
13,неоконченное высшее,2
31,начальное,3
2962,ученая степень,4


In [43]:
family_status_dict = data[['family_status', 'family_status_id']]
family_status_dict.drop_duplicates()

,family_status,family_status_id
0,женат / замужем,0
4,гражданский брак,1
18,вдовец / вдова,2
19,в разводе,3
24,не женат / не замужем,4


In [44]:
data.pop('education')
data.pop('family_status')
data.head()

,index,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose
0,0,1,8437.673028,42,0,0,F,сотрудник,0,253875,покупка жилья
1,1,1,4024.803754,36,1,0,F,сотрудник,0,112080,приобретение автомобиля
2,2,0,5623.422610,33,1,0,M,сотрудник,0,145885,покупка жилья
3,3,3,4124.747207,32,1,0,M,сотрудник,0,267628,дополнительное образование
4,4,0,14177.753002,53,1,1,F,пенсионер,0,158616,сыграть свадьбу


### Категоризация дохода.

Создадим столбец с категориями доходов по заданным диапазонам

In [45]:
def category_total_income(row):
    income = row['total_income']
    if 0 < income < 30000:
        return 'E'
    if 30001 < income < 50000:
        return 'D'
    if 50001 < income < 200000:
        return 'C'
    if 200001 < income < 1000000:
        return 'B'
    return 'A'

data['total_income_category'] = data.apply(category_total_income, axis=1)
data.head()

,index,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category
0,0,1,8437.673028,42,0,0,F,сотрудник,0,253875,покупка жилья,B
1,1,1,4024.803754,36,1,0,F,сотрудник,0,112080,приобретение автомобиля,C
2,2,0,5623.422610,33,1,0,M,сотрудник,0,145885,покупка жилья,C
3,3,3,4124.747207,32,1,0,M,сотрудник,0,267628,дополнительное образование,B
4,4,0,14177.753002,53,1,1,F,пенсионер,0,158616,сыграть свадьбу,C


###  Категоризация целей кредита.

Сформируем новый столбец с категориями целей кредита

In [46]:
def category_purpose(row):
    category = row['purpose']
    if 'автомобил' in category:
        return 'операции с автомобилем'
    if ('недвижимост' or 'жил') in category:
        return 'операции с недвижимостью'
    if 'свадьб' in category:
        return 'проведение свадьбы'
    if 'образовани' in category:
        return 'получение образования'
    
data['purpose_category'] = data.apply(category_purpose, axis=1)
data.head()    


,index,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category,purpose_category
0,0,1,8437.673028,42,0,0,F,сотрудник,0,253875,покупка жилья,B,None
1,1,1,4024.803754,36,1,0,F,сотрудник,0,112080,приобретение автомобиля,C,операции с автомобилем
2,2,0,5623.422610,33,1,0,M,сотрудник,0,145885,покупка жилья,C,None
3,3,3,4124.747207,32,1,0,M,сотрудник,0,267628,дополнительное образование,B,получение образования
4,4,0,14177.753002,53,1,1,F,пенсионер,0,158616,сыграть свадьбу,C,проведение свадьбы


## Ответы на вопросы.

##### Вопрос 1: Есть ли зависимость между количеством детей и возвратом кредита в срок?

Создадим столбец с категориями по количеству детей 

In [47]:
def category_children(row):
    child = row['children']
    if child == 0:
        return 'нет детей'
    if 0 < child < 3:
        return '1-2 ребенка'
    return 'многодетные'

data['category_children'] = data.apply(category_children, axis=1)

In [48]:
data_pivot = data.pivot_table(index='category_children', columns='debt', values='children', aggfunc='count')
data_pivot

debt,0,1
category_children,,
1-2 ребенка,6336,647
многодетные,349,31
нет детей,13028,1063


In [49]:
data_pivot['ratio'] =  data_pivot[1] / (data_pivot[0] + data_pivot[1])
data_pivot.sort_values(by='ratio')

debt,0,1,ratio
category_children,,,
нет детей,13028,1063,0.075438
многодетные,349,31,0.081579
1-2 ребенка,6336,647,0.092654


##### Вывод 1:

Зависимость есть. В категории "нет детей" число людей с задолженностью составляет 7%, а в категории "1-2 ребенка" - 9%

##### Вопрос 2: Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [50]:
data_pivot = data.pivot_table(index=family_status_dict['family_status'], columns='debt', values='family_status_id', aggfunc='count')
data_pivot

debt,0,1
family_status,,
в разводе,1110,85
вдовец / вдова,896,63
гражданский брак,3763,388
женат / замужем,11408,931
не женат / не замужем,2536,274


In [51]:
data_pivot['ratio'] =  data_pivot[1] / (data_pivot[0] + data_pivot[1])
data_pivot.sort_values(by='ratio')

debt,0,1,ratio
family_status,,,
вдовец / вдова,896,63,0.065693
в разводе,1110,85,0.071130
женат / замужем,11408,931,0.075452
гражданский брак,3763,388,0.093471
не женат / не замужем,2536,274,0.097509


##### Вывод 2:

Зависимость есть. В категории "вдовец/вдова" наименьший процент должников, тогда как в категории "не женат/не замужем" он составляет почти 10%

##### Вопрос 3: Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [52]:
data_pivot = data.pivot_table(index=['total_income_category'], columns='debt', values='total_income', aggfunc='count')
data_pivot

debt,0,1
total_income_category,,
A,24,2
B,4685,356
C,14655,1360
D,329,21
E,20,2


In [53]:
data_pivot['ratio'] = data_pivot[1] / (data_pivot[0] + data_pivot[1])
data_pivot.sort_values(by='ratio')

debt,0,1,ratio
total_income_category,,,
D,329,21,0.060000
B,4685,356,0.070621
A,24,2,0.076923
C,14655,1360,0.084920
E,20,2,0.090909


##### Вывод 3:

Вероятно, зависимости нет, так как люди, чьи доходы ниже среднего(категория E и D), имеют наибольший(9%) и наименьший(6%) процент задолженности по кредитам. Но при этом люди с доходами выше среднего все-таки имеют меньший процент задолженностей, чем люди со средним уровнем дохода.
*средний уровень - категория C

##### Вопрос 4: Как разные цели кредита влияют на его возврат в срок?

In [54]:
data_pivot = data.pivot_table(index=['purpose_category'], columns='debt', values='purpose', aggfunc='count')
data_pivot

debt,0,1
purpose_category,,
операции с автомобилем,3903,403
операции с недвижимостью,5877,474
получение образования,3643,370
проведение свадьбы,2138,186


In [55]:
data_pivot['ratio'] = data_pivot[1] / (data_pivot[0] + data_pivot[1])
data_pivot.sort_values(by='ratio')

debt,0,1,ratio
purpose_category,,,
операции с недвижимостью,5877,474,0.074634
проведение свадьбы,2138,186,0.080034
получение образования,3643,370,0.092200
операции с автомобилем,3903,403,0.093590


##### Вывод 4:

Зависимость есть. Больший процент задолженностей в категориях "образование" и "автомобиль" - 9%.

## Общий вывод:

В ходе исследования мы сделали следующее:
* проверили данные на наличие пропусков и заполнили их медианными значениями
* выявили аномалии в столбцах "стаж работы", "пол", "возраст" и "количество детей" 
* выявили явные и не явные дубликаты и избавились от них
* категоризировали доходы, цели кредита и количество детей 

На основании имеющихся данных выявили:

* Количество детей может влиять на факт погашения кредита в срок. Доля людей, у которых нет детей, меньше, чем доля тех, у кого 1-2 ребенка
* Семейное положение также может влиять на факт погашения кредита в срок. Так, овдовешие люди имеют наименьший процент задолженностей (около 7%), а у людей, не состоящих в браке вообще или состоящих в гражданском браке, наибольший процент задолженностей( в обеих категориях - около 10%)
* Можно сказать, что зависимость между уровнем дохода и возвратом кредита в срок есть, если уровень дохода выше среднего. У таких людей процент задолженностей около 7%. При этом люди с доходами ниже среднего имеют как наименьший (6%), так и наибольший (9%) процент по задолженностям. То есть нельзя сказать наверняка, будут ли люди с таким уровнем дохода выплачивать кредит в срок.
* Наибольший процент должников среди людей, берущих кредит на получение образования или операции с автомобилем - 9% и 10% соответсвенно. 
